<a href="https://colab.research.google.com/github/Vijayputta007/Bank_transactions/blob/main/HR_analytics_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 45.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=366de848b465415c4cbd1c2cb403a3a77c9f168435f0cf242cd894e5db687a2f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pyspark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HR Analytics').getOrCreate()

In [6]:
hrtrain = spark.read.csv('HRtrain.csv',inferSchema=True, header=True)

In [8]:
hrtest= spark.read.csv('HRtest.csv',inferSchema=True, header=True)

In [9]:
hrtrain.show(5)

+-----------+-----------------+---------+----------------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+-----------+
|employee_id|       department|   region|       education|gender|recruitment_channel|no_of_trainings|age|previous_year_rating|length_of_service|KPIs_met >80%|awards_won?|avg_training_score|is_promoted|
+-----------+-----------------+---------+----------------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+-----------+
|      65438|Sales & Marketing| region_7|Master's & above|     f|           sourcing|              1| 35|                   5|                8|            1|          0|                49|          0|
|      65141|       Operations|region_22|      Bachelor's|     m|              other|              1| 30|                   5|                4|            0|          0|                60|   

In [10]:
from pyspark.sql.functions import isnull,when,count,col

In [13]:
hrtrain.select([count(when(isnull(c),c)).alias(c) for c in hrtrain.columns]).show()

+-----------+----------+------+---------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+-----------+
|employee_id|department|region|education|gender|recruitment_channel|no_of_trainings|age|previous_year_rating|length_of_service|KPIs_met >80%|awards_won?|avg_training_score|is_promoted|
+-----------+----------+------+---------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+-----------+
|          0|         0|     0|     2409|     0|                  0|              0|  0|                4124|                0|            0|          0|                 0|          0|
+-----------+----------+------+---------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+-----------+



In [14]:
hrtest.select([count(when(isnull(c),c)).alias(c) for c in hrtest.columns]).show()

+-----------+----------+------+---------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+
|employee_id|department|region|education|gender|recruitment_channel|no_of_trainings|age|previous_year_rating|length_of_service|KPIs_met >80%|awards_won?|avg_training_score|
+-----------+----------+------+---------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+
|          0|         0|     0|     1034|     0|                  0|              0|  0|                1812|                0|            0|          0|                 0|
+-----------+----------+------+---------+------+-------------------+---------------+---+--------------------+-----------------+-------------+-----------+------------------+



In [15]:
hrtrain.groupBy('education').count().show()

+----------------+-----+
|       education|count|
+----------------+-----+
|            null| 2409|
| Below Secondary|  805|
|Master's & above|14925|
|      Bachelor's|36669|
+----------------+-----+



In [16]:
hrtest.groupBy('education').count().show()

+----------------+-----+
|       education|count|
+----------------+-----+
|            null| 1034|
| Below Secondary|  374|
|Master's & above| 6504|
|      Bachelor's|15578|
+----------------+-----+



In [18]:
hrtrain = hrtrain.na.fill(value="Bachelor's",subset=['education'])

In [19]:
hrtest= hrtest.na.fill(value="Bachelor's",subset=['education'])

In [21]:
hrtrain.groupBy('previous_year_rating').count().show()

+--------------------+-----+
|previous_year_rating|count|
+--------------------+-----+
|                null| 4124|
|                   1| 6223|
|                   3|18618|
|                   5|11741|
|                   4| 9877|
|                   2| 4225|
+--------------------+-----+



In [22]:
hrtrain=hrtrain.na.fill(value=3,subset=['previous_year_rating'])

In [23]:
hrtest=hrtest.na.fill(value=3,subset=['previous_year_rating'])

In [24]:
hrtrain.columns

['employee_id',
 'department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met >80%',
 'awards_won?',
 'avg_training_score',
 'is_promoted']

In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [31]:
indexer= [StringIndexer(inputCol=col,outputCol=col+'_index').fit(hrtrain)
for col in list(set(hrtrain.columns)-set(['avg_training_score','length_of_service','age','no_of_trainings','employee_id']))]

In [35]:
pipeline = Pipeline(stages=indexer)

In [36]:
hrtraindf=pipeline.fit(hrtrain).transform(hrtrain)

In [38]:
hrtestdf=pipeline.fit(hrtest).transform(hrtest)

In [39]:
hrtraindf.columns

['employee_id',
 'department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met >80%',
 'awards_won?',
 'avg_training_score',
 'is_promoted',
 'department_index',
 'previous_year_rating_index',
 'recruitment_channel_index',
 'education_index',
 'awards_won?_index',
 'KPIs_met >80%_index',
 'gender_index',
 'region_index',
 'is_promoted_index']

In [40]:
columnstodrop=['employee_id','department','region','education','gender',
               'recruitment_channel','previous_year_rating','KPIs_met >80%',
               'awards_won?','is_promoted']

In [43]:
hrtraindf=hrtraindf.drop(*columnstodrop)

In [44]:
hrtestdf=hrtestdf.drop(*columnstodrop)

In [45]:
from pyspark.ml.feature import RFormula

In [52]:
formula = RFormula(formula='is_promoted_index~.',featuresCol='features',labelCol='label')

In [53]:
hrtraindf=formula.fit(hrtraindf).transform(hrtraindf)

In [54]:
hrtraindf.select('features','label').show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,35.0,8.0,49....|  0.0|
|[1.0,30.0,4.0,60....|  0.0|
|(12,[0,1,2,3,6,11...|  0.0|
|(12,[0,1,2,3,5,11...|  0.0|
|(12,[0,1,2,3,4,11...|  0.0|
+--------------------+-----+
only showing top 5 rows



In [56]:
from pyspark.ml.classification import LogisticRegression

In [57]:
logit = LogisticRegression(featuresCol='features',labelCol='label')

In [58]:
logitmodel=logit.fit(hrtraindf)

In [59]:
logitmodel.summary.accuracy

0.9209057071960298

In [61]:
logitmodel.summary.areaUnderROC

0.8074795905654398

In [62]:
hrtestdf.columns

['no_of_trainings',
 'age',
 'length_of_service',
 'avg_training_score',
 'department_index',
 'previous_year_rating_index',
 'recruitment_channel_index',
 'education_index',
 'awards_won?_index',
 'KPIs_met >80%_index',
 'gender_index',
 'region_index']

In [64]:
from pyspark.ml.feature import VectorAssembler

In [67]:
assembler=VectorAssembler(inputCols=['no_of_trainings','age',
 'length_of_service',
 'avg_training_score',
 'department_index',
 'previous_year_rating_index',
 'recruitment_channel_index',
 'education_index',
 'awards_won?_index',
 'KPIs_met >80%_index',
 'gender_index',
 'region_index'],outputCol='features')

In [68]:
hrtestdf=assembler.transform(hrtestdf)

In [69]:
hrtestdf.show()

+---------------+---+-----------------+------------------+----------------+--------------------------+-------------------------+---------------+-----------------+-------------------+------------+------------+--------------------+
|no_of_trainings|age|length_of_service|avg_training_score|department_index|previous_year_rating_index|recruitment_channel_index|education_index|awards_won?_index|KPIs_met >80%_index|gender_index|region_index|            features|
+---------------+---+-----------------+------------------+----------------+--------------------------+-------------------------+---------------+-----------------+-------------------+------------+------------+--------------------+
|              1| 24|                1|                77|             3.0|                       0.0|                      1.0|            0.0|              0.0|                1.0|         0.0|         5.0|[1.0,24.0,1.0,77....|
|              1| 31|                5|                51|             6.0|     

In [70]:
logitpredict=logitmodel.transform(hrtestdf)

In [73]:
logitpredict.select('prediction').toPandas().to_csv('logit.csv')

In [74]:
from pyspark.ml.classification import DecisionTreeClassifier

In [77]:
tree = DecisionTreeClassifier(featuresCol='features',labelCol='label',maxBins=35)

In [78]:
treemodel = tree.fit(hrtraindf)

In [79]:
treemodel=treemodel.transform(hrtraindf)

In [80]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [84]:
evaluator= MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label',metricName='accuracy')

In [85]:
evaluator.evaluate(treemodel)

0.9275653189315428

In [86]:
from pyspark.ml.classification import RandomForestClassifier

In [90]:
RF=RandomForestClassifier(featuresCol='features',labelCol='label',maxBins=35,numTrees=1000)

In [91]:
RFmodel= RF.fit(hrtraindf)

In [92]:
RFmodel= RFmodel.transform(hrtraindf)

In [93]:
evaluator.evaluate(RFmodel)

0.9260144504451905

In [95]:
from pyspark.ml.classification import GBTClassifier

In [96]:
gbm = GBTClassifier(featuresCol='features', labelCol='label',maxBins=35)

In [97]:
gbmmodel= gbm.fit(hrtraindf)

In [98]:
gbmmodel=gbmmodel= gbmmodel.transform(hrtraindf)

In [99]:
evaluator.evaluate(gbmmodel)

0.9416873449131513

In [100]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [109]:
nn= MultilayerPerceptronClassifier(featuresCol='features',labelCol='label',layers=[12,100,50,2])

In [110]:
nnmodel = nn.fit(hrtraindf)

In [111]:
nnmodel=nnmodel.transform(hrtraindf)

In [112]:
evaluator.evaluate(nnmodel)

0.9185702817106992

In [113]:
from pyspark.ml.classification import NaiveBayes

In [114]:
nb= NaiveBayes(featuresCol='features',labelCol='label')

In [116]:
nbmodel= nb.fit(hrtraindf)

In [117]:
nbmodel= nbmodel.transform(hrtraindf)

In [118]:
evaluator.evaluate(nbmodel)

0.8809845278061597